# 🪙 Detector de Monedas - Inspector de Calidad Casero

Este notebook implementa el proyecto completo:
1. **Preparación del dataset** (formato COCO desde CVAT)
2. **Fine-tuning de DETR** con Hugging Face
3. **Evaluación del modelo**
4. **Exportación a ONNX**

⚠️ **Recomendación**: Ejecutar en Google Colab con GPU habilitada

## 📦 1. Instalación de Dependencias

In [ ]:
# Instalación de librerías necesarias
# Descomentar si estás en Google Colab

# !pip install -q transformers datasets torch torchvision
# !pip install -q pycocotools albumentations
# !pip install -q onnx onnxruntime
# !pip install -q timm accelerate

In [ ]:
# Importaciones
import os
import json
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path

# Hugging Face
from transformers import DetrImageProcessor, DetrForObjectDetection
from transformers import TrainingArguments, Trainer

# PyTorch Dataset
from torch.utils.data import Dataset, DataLoader

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 📁 2. Configuración del Dataset

### Estructura esperada de carpetas:
```
dataset/
├── train/
│   ├── images/
│   │   ├── img_001.jpg
│   │   └── ...
│   └── annotations.json  (formato COCO exportado de CVAT)
└── val/
    ├── images/
    └── annotations.json
```

In [ ]:
# ============================================
# 🔧 CONFIGURACIÓN - MODIFICAR SEGÚN TU CASO
# ============================================

# Rutas al dataset (cambiar según tu estructura)
DATASET_PATH = "./dataset"  # Carpeta raíz del dataset
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "val")

# Clases del proyecto
CLASSES = ["cara", "cruz"]  # Tus etiquetas de CVAT
NUM_CLASSES = len(CLASSES)

# Hiperparámetros de entrenamiento
BATCH_SIZE = 4
NUM_EPOCHS = 30
LEARNING_RATE = 1e-5

# Modelo base
MODEL_NAME = "facebook/detr-resnet-50"

print(f"Clases: {CLASSES}")
print(f"Número de clases: {NUM_CLASSES}")

## 📊 3. Cargar y Explorar el Dataset COCO

In [ ]:
def load_coco_annotations(annotation_path):
    """Carga las anotaciones COCO exportadas de CVAT."""
    with open(annotation_path, 'r') as f:
        coco_data = json.load(f)
    
    print(f"📊 Estadísticas del dataset:")
    print(f"   - Imágenes: {len(coco_data['images'])}")
    print(f"   - Anotaciones: {len(coco_data['annotations'])}")
    print(f"   - Categorías: {[cat['name'] for cat in coco_data['categories']]}")
    
    return coco_data

# Cargar anotaciones (descomentar cuando tengas el dataset)
# train_coco = load_coco_annotations(os.path.join(TRAIN_PATH, "annotations.json"))

In [ ]:
def visualize_sample(coco_data, images_path, idx=0):
    """Visualiza una imagen con sus anotaciones."""
    # Obtener imagen
    img_info = coco_data['images'][idx]
    img_path = os.path.join(images_path, img_info['file_name'])
    img = Image.open(img_path)
    
    # Obtener anotaciones de esta imagen
    img_id = img_info['id']
    annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == img_id]
    
    # Crear mapeo de categorías
    cat_map = {cat['id']: cat['name'] for cat in coco_data['categories']}
    
    # Visualizar
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)
    
    colors = {'cara': 'green', 'cruz': 'red'}
    
    for ann in annotations:
        bbox = ann['bbox']  # [x, y, width, height] en formato COCO
        cat_name = cat_map[ann['category_id']]
        color = colors.get(cat_name, 'blue')
        
        rect = patches.Rectangle(
            (bbox[0], bbox[1]), bbox[2], bbox[3],
            linewidth=2, edgecolor=color, facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(bbox[0], bbox[1] - 5, cat_name, color=color, fontsize=12, fontweight='bold')
    
    ax.set_title(f"Imagen: {img_info['file_name']}")
    ax.axis('off')
    plt.show()

# Visualizar ejemplo (descomentar cuando tengas datos)
# visualize_sample(train_coco, os.path.join(TRAIN_PATH, "images"), idx=0)

## 🔄 4. Dataset de PyTorch para DETR

In [ ]:
class CocoDetectionDataset(Dataset):
    """
    Dataset personalizado para cargar datos COCO y prepararlos para DETR.
    """
    def __init__(self, images_path, annotation_path, processor):
        self.images_path = images_path
        self.processor = processor
        
        # Cargar anotaciones COCO
        with open(annotation_path, 'r') as f:
            self.coco_data = json.load(f)
        
        self.images = self.coco_data['images']
        self.annotations = self.coco_data['annotations']
        
        # Crear índice de anotaciones por imagen
        self.img_to_anns = {}
        for ann in self.annotations:
            img_id = ann['image_id']
            if img_id not in self.img_to_anns:
                self.img_to_anns[img_id] = []
            self.img_to_anns[img_id].append(ann)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Cargar imagen
        img_info = self.images[idx]
        img_path = os.path.join(self.images_path, img_info['file_name'])
        image = Image.open(img_path).convert("RGB")
        
        # Obtener anotaciones
        img_id = img_info['id']
        annotations = self.img_to_anns.get(img_id, [])
        
        # Preparar target en formato DETR
        boxes = []
        labels = []
        
        for ann in annotations:
            # COCO format: [x, y, width, height] -> convertir a [x_center, y_center, w, h] normalizado
            x, y, w, h = ann['bbox']
            # Normalizar por dimensiones de la imagen
            img_w, img_h = image.size
            x_center = (x + w / 2) / img_w
            y_center = (y + h / 2) / img_h
            w_norm = w / img_w
            h_norm = h / img_h
            
            boxes.append([x_center, y_center, w_norm, h_norm])
            labels.append(ann['category_id'] - 1)  # DETR espera labels desde 0
        
        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "class_labels": torch.tensor(labels, dtype=torch.long),
            "image_id": torch.tensor([img_id])
        }
        
        # Procesar con el processor de DETR
        encoding = self.processor(
            images=image,
            annotations={"boxes": target["boxes"].tolist(), "class_labels": target["class_labels"].tolist()},
            return_tensors="pt"
        )
        
        # Quitar dimensión batch
        pixel_values = encoding["pixel_values"].squeeze(0)
        labels = encoding["labels"][0]
        
        return {"pixel_values": pixel_values, "labels": labels}

## 🧠 5. Cargar Modelo DETR Pre-entrenado

In [ ]:
# Cargar processor y modelo
processor = DetrImageProcessor.from_pretrained(MODEL_NAME)

# Cargar modelo con número de clases personalizado
model = DetrForObjectDetection.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_CLASSES,
    ignore_mismatched_sizes=True  # Importante para cambiar el número de clases
)

print(f"✅ Modelo cargado: {MODEL_NAME}")
print(f"   - Número de parámetros: {sum(p.numel() for p in model.parameters()):,}")
print(f"   - Parámetros entrenables: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 🏋️ 6. Entrenamiento

In [ ]:
def collate_fn(batch):
    """Función para agrupar muestras en un batch."""
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = [item["labels"] for item in batch]
    return {"pixel_values": pixel_values, "labels": labels}

# Crear datasets (descomentar cuando tengas los datos)
# train_dataset = CocoDetectionDataset(
#     images_path=os.path.join(TRAIN_PATH, "images"),
#     annotation_path=os.path.join(TRAIN_PATH, "annotations.json"),
#     processor=processor
# )
# 
# val_dataset = CocoDetectionDataset(
#     images_path=os.path.join(VAL_PATH, "images"),
#     annotation_path=os.path.join(VAL_PATH, "annotations.json"),
#     processor=processor
# )
# 
# print(f"Train samples: {len(train_dataset)}")
# print(f"Val samples: {len(val_dataset)}")

In [ ]:
# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./detr_monedas_output",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=1e-4,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    evaluation_strategy="steps",
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    fp16=torch.cuda.is_available(),  # Mixed precision si hay GPU
)

print("✅ Configuración de entrenamiento lista")

In [ ]:
# Crear Trainer (descomentar cuando tengas datasets)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=collate_fn,
# )

# Entrenar
# print("🚀 Iniciando entrenamiento...")
# trainer.train()
# print("✅ Entrenamiento completado!")

## 🔍 7. Inferencia - Probar el Modelo

In [ ]:
def predict_and_visualize(model, processor, image_path, threshold=0.7):
    """
    Realiza predicción en una imagen y visualiza los resultados.
    
    Args:
        model: Modelo DETR entrenado
        processor: DetrImageProcessor
        image_path: Ruta a la imagen
        threshold: Umbral de confianza para mostrar predicciones
    """
    # Cargar imagen
    image = Image.open(image_path).convert("RGB")
    
    # Preprocesar
    inputs = processor(images=image, return_tensors="pt")
    
    # Mover a GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Inferencia
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Post-procesar resultados
    target_sizes = torch.tensor([image.size[::-1]]).to(device)
    results = processor.post_process_object_detection(
        outputs, 
        target_sizes=target_sizes,
        threshold=threshold
    )[0]
    
    # Visualizar
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)
    
    colors = {0: 'green', 1: 'red'}  # cara=0, cruz=1
    
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = box.cpu().numpy()
        x1, y1, x2, y2 = box
        
        label_idx = label.item()
        label_name = CLASSES[label_idx]
        confidence = score.item()
        color = colors.get(label_idx, 'blue')
        
        # Dibujar caja
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=3, edgecolor=color, facecolor='none'
        )
        ax.add_patch(rect)
        
        # Etiqueta con confianza
        ax.text(
            x1, y1 - 10, 
            f"{label_name}: {confidence:.2f}", 
            color='white', fontsize=12, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor=color, alpha=0.8)
        )
    
    ax.set_title(f"Predicciones (umbral={threshold})")
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Imprimir resultados
    print(f"\n📊 Detecciones encontradas: {len(results['scores'])}")
    for i, (score, label) in enumerate(zip(results["scores"], results["labels"])):
        print(f"   {i+1}. {CLASSES[label.item()]}: {score.item():.2%}")

# Ejemplo de uso (descomentar con una imagen real)
# predict_and_visualize(model, processor, "./test_image.jpg", threshold=0.5)

## 📦 8. Exportar a ONNX

In [ ]:
def export_to_onnx(model, output_path="detector_monedas.onnx", input_size=(800, 800)):
    """
    Exporta el modelo DETR a formato ONNX.
    
    Args:
        model: Modelo DETR entrenado
        output_path: Ruta de salida del archivo .onnx
        input_size: Tamaño de entrada (alto, ancho)
    """
    print("🔄 Preparando modelo para exportación ONNX...")
    
    # Poner modelo en modo evaluación y CPU
    model.eval()
    model.to("cpu")
    
    # Crear entrada dummy
    dummy_input = torch.randn(1, 3, input_size[0], input_size[1])
    
    print(f"   - Input shape: {dummy_input.shape}")
    
    # Exportar
    torch.onnx.export(
        model,
        dummy_input,
        output_path,
        opset_version=12,
        input_names=["pixel_values"],
        output_names=["logits", "pred_boxes"],
        dynamic_axes={
            "pixel_values": {0: "batch_size"},
            "logits": {0: "batch_size"},
            "pred_boxes": {0: "batch_size"}
        },
        do_constant_folding=True
    )
    
    # Verificar tamaño del archivo
    file_size = os.path.getsize(output_path) / (1024 * 1024)
    
    print(f"\n✅ Modelo exportado exitosamente!")
    print(f"   - Archivo: {output_path}")
    print(f"   - Tamaño: {file_size:.2f} MB")
    
    return output_path

# Exportar (descomentar cuando tengas el modelo entrenado)
# onnx_path = export_to_onnx(model, "detector_monedas.onnx")

In [ ]:
def verify_onnx_model(onnx_path):
    """
    Verifica que el modelo ONNX es válido.
    """
    import onnx
    
    print(f"🔍 Verificando modelo ONNX: {onnx_path}")
    
    # Cargar y verificar
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    
    # Imprimir información
    print(f"\n✅ Modelo ONNX válido!")
    print(f"\n📊 Información del modelo:")
    print(f"   - IR Version: {onnx_model.ir_version}")
    print(f"   - Opset Version: {onnx_model.opset_import[0].version}")
    
    # Inputs
    print(f"\n   Inputs:")
    for inp in onnx_model.graph.input:
        print(f"      - {inp.name}: {[dim.dim_value for dim in inp.type.tensor_type.shape.dim]}")
    
    # Outputs
    print(f"\n   Outputs:")
    for out in onnx_model.graph.output:
        print(f"      - {out.name}")

# Verificar (descomentar después de exportar)
# verify_onnx_model("detector_monedas.onnx")

## ⚡ 9. Inferencia con ONNX Runtime

In [ ]:
def inference_with_onnx(onnx_path, image_path, processor, threshold=0.5):
    """
    Realiza inferencia usando ONNX Runtime (más rápido que PyTorch).
    """
    import onnxruntime as ort
    
    print(f"🔄 Cargando modelo ONNX...")
    
    # Crear sesión de inferencia
    session = ort.InferenceSession(onnx_path)
    
    # Cargar y preprocesar imagen
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="np")
    
    # Inferencia
    print(f"🚀 Ejecutando inferencia...")
    outputs = session.run(
        None,
        {"pixel_values": inputs["pixel_values"]}
    )
    
    logits, pred_boxes = outputs
    
    # Post-procesar
    probas = torch.softmax(torch.tensor(logits), dim=-1)[0, :, :-1]  # Excluir clase "no object"
    keep = probas.max(-1).values > threshold
    
    print(f"\n✅ Inferencia completada!")
    print(f"   - Detecciones sobre umbral {threshold}: {keep.sum().item()}")
    
    return outputs

# Ejemplo (descomentar cuando tengas el modelo)
# inference_with_onnx("detector_monedas.onnx", "./test_image.jpg", processor)

## 📋 10. Checklist del Proyecto

### Sábado - Datos ✅
- [ ] Capturar 30-40 fotos de monedas
- [ ] Crear cuenta en CVAT.ai
- [ ] Subir imágenes y crear Task
- [ ] Definir labels: `cara`, `cruz`
- [ ] Anotar todas las imágenes con bounding boxes
- [ ] Exportar en formato COCO 1.0
- [ ] Dividir en train (80%) y val (20%)

### Domingo - Modelo ✅
- [ ] Subir notebook a Google Colab
- [ ] Subir dataset a Colab
- [ ] Ejecutar entrenamiento (20-50 épocas)
- [ ] Evaluar en imágenes de validación
- [ ] Exportar a ONNX
- [ ] Verificar modelo ONNX
- [ ] Probar inferencia con ONNX Runtime

In [ ]:
# Guardar el modelo entrenado (descomentar cuando entrenes)
# model.save_pretrained("./detr_monedas_final")
# processor.save_pretrained("./detr_monedas_final")
# print("✅ Modelo guardado en ./detr_monedas_final")

---

## 🎤 Resumen para la Entrevista

> *"Desarrollé un pipeline completo de visión artificial: capturé y anoté mi propio dataset en **CVAT** (formato COCO), realicé fine-tuning de un modelo **DETR** usando Hugging Face Transformers para detectar objetos pequeños con precisión, y finalmente exporté el modelo a **ONNX** para optimizar la inferencia en producción. El proyecto demuestra mi capacidad para manejar el ciclo completo de un sistema de ML, desde la anotación de datos hasta el despliegue."*